In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from PIL import Image
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np
from torchvision.models import resnet50
from torch.autograd import Variable
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define transformations for training dataset (with data augmentation)
transform = transforms.Compose([
    #transforms.RandomRotation(15),  # Rotation by ±15 degrees
    #transforms.RandomHorizontalFlip(),  # Horizontal flip
    #transforms.RandomVerticalFlip(),  # Uncomment if vertical flip is desired
    transforms.ToTensor(),  # Convert images to tensor
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

# Load CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class CNNLateralConnections(nn.Module):
    def __init__(self, dropout_rate=0.5, num_classes=10):
        super(CNNLateralConnections, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=5, stride=1, padding=2)  # Added padding to maintain size
        self.bn1 = nn.BatchNorm2d(num_features=96)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)  # Added padding to maintain size
        self.bn2 = nn.BatchNorm2d(num_features=256)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.conv3 = nn.Conv2d(in_channels=352, out_channels=384, kernel_size=3, stride=1, padding=1)  # Input channels must match the sum from concatenated layers
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=1120, out_channels=256, kernel_size=3, stride=1, padding=1)  # Adjusted to accommodate all concatenated layers
        self.dropout2 = nn.Dropout(p=dropout_rate)
        self.fc1 = nn.Linear(in_features=65536, out_features=num_classes)  # Adjusted size based on flattened dimensions from final conv layer

    def forward(self, x):
        x1 = F.relu(self.bn1(self.conv1(x)))  # Output size: [96, 224, 224]
        x2 = F.relu(self.bn2(self.conv2(x1)))  # Output size: [256, 224, 224]
        x2_pooled = self.pool1(x2)  # Output size: [256, 112, 112]

        x1_resized = F.adaptive_avg_pool2d(x1, x2_pooled.shape[2:])  # Resize x1 to match x2_pooled dimensions
        x3_input = torch.cat([x1_resized, x2_pooled], dim=1)  # Channel dimension: 96 + 256 = 352
        x3 = F.relu(self.conv3(x3_input))  # Output size: [384, 112, 112]
        x4 = F.relu(self.conv4(x3))  # Output size: [384, 112, 112]

        x2_resized = F.adaptive_avg_pool2d(x2, x4.shape[2:])  # Resize x2 to match x4 dimensions
        x5_input = torch.cat([x4, x1_resized, x2_resized, x4], dim=1)  # Channel dimension: 384 + 96 + 256 + 384 = 1120
        x5 = F.relu(self.conv5(x5_input))  # Output size: [256, 112, 112]
        x5 = x5.view(x5.size(0), -1)  # Flatten the tensor

        x5 = self.dropout2(x5)  # Apply dropout before the fully connected layer
        #print(x5.size())
        output = self.fc1(x5)  # Final output
        return output

In [ ]:
# Training loop
def train(epoch, net, trainloader, optimizer, criterion):
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    # Calculate accuracy after each epoch
    accuracy = 100 * correct / total
    epoch_list.append(epoch + 1)  # Append epoch number to list
    accuracy_list.append(accuracy)  # Append accuracy to list
    print('Epoch [%d], Accuracy on training images: %d %%' % (epoch + 1, accuracy))

In [ ]:
# Test the model
def test(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)  # Move data to GPU
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy on test images: %d %%' % (100 * correct / total))

In [ ]:
# Iniltialize the model
net = CNNLateralConnections()
net.to(device)

#Initialize optimizer and Loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Lists to store epoch and accuracy values
epoch_list = []
accuracy_list = []
# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    train(epoch, net, trainloader, optimizer, criterion)

# Test the model
test(net, testloader)

Epoch [1], Accuracy on training images: 48 %
Epoch [2], Accuracy on training images: 63 %
Epoch [3], Accuracy on training images: 69 %
Epoch [4], Accuracy on training images: 73 %
Epoch [5], Accuracy on training images: 76 %
Epoch [6], Accuracy on training images: 79 %
Epoch [7], Accuracy on training images: 81 %
Epoch [8], Accuracy on training images: 83 %
Epoch [9], Accuracy on training images: 85 %
Epoch [10], Accuracy on training images: 87 %
Accuracy on test images: 77 %


In [ ]:
# Plot epoch vs accuracy
plt.plot(epoch_list, accuracy_list, marker='o')
plt.title('Epoch vs Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True)
plt.show()